In [ ]:
import stft
import peaks
import os
import scipy.io.wavfile as wav
import IPython.display as ipd
import ipywidgets as ipw

In [ ]:
AUDIO_DIR = "../audio"
wavfiles = os.listdir(AUDIO_DIR)

def main(filename, fft_length, freq_thresh, note_gap_time, min_note_length, peak_height):
    fs, data = wav.read(f"{AUDIO_DIR}/{filename}")
    if len(data.shape) == 2:
        data = data[:, 0] # get just one channel

    if max(data) < 1.01:
        data = data * 2**15 # convert [-1, 1] to [-2^15, 2^15] 

    ipd.display(ipd.Audio(data, rate=fs))
    
    Zxx, f, t = stft.analysis(data, N=int(fft_length))
    stft.spectrogram(Zxx, f, t)

    peaks.plot_peaks(Zxx, f, t, height=peak_height)
    notes = peaks.find_notes(Zxx, f, t, freq_thresh, note_gap_time, min_note_length, height=peak_height)
    note_reprs = repr(notes)[1:-1].split(", ")
    print(f"Detected {len(notes)} notes in the signal:", *note_reprs, sep="\n    ")
    peaks.plot_notes(notes)

ipw.interact(main,
             filename=wavfiles,
             fft_length=ipw.FloatLogSlider(value=1024, base=2, min=8, max=12, step=1),
             freq_thresh=ipw.FloatSlider(value=30, min=10, max=100), 
             note_gap_time=ipw.FloatSlider(value=0.05, min=0.05, max=0.5, step=0.05),
             min_note_length=ipw.FloatSlider(value=0.5, min=0.01, max=1, step=0.01),
             peak_height=ipw.FloatLogSlider(value=150, min=1, max=3, step=0.1))